# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [15]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,MLSS-Indonesia 2020 Machine Learning Tutorial 1,Tutorial,mlss-indo-2020-tutorial-1,"Telkom University, Bandung",20-08-03,"Bandung, Indonesia",https://mlss.telkomuniversity.ac.id/,MLSS-Indo 2020 - Practical 1 - Machine Learnin...
1,MLSS-Indonesia 2020 Machine Learning Tutorial 5,Tutorial,mlss-indo-2020-tutorial-5,"Telkom University, Bandung",20-08-04,"Bandung, Indonesia",https://mlss.telkomuniversity.ac.id/,MLSS-Indo 2020 - Practical 5 - Variational Aut...
2,ICoICT 2018,Conference proceedings talk,icoict-2018-talk,"Telkom University, Bandung",18-05-03,"Bandung, Indonesia",https://2018.icoict.org/,Endorsement Recommendation using Instagram Fol...
3,ICoICT 2019,Conference proceedings talk,icoict-2019-talk,"The Vederal Hotel, Kuala Lumpur",19-07-24,"Kuala Lumpur, Malaysia",https://2019.icoict.org/,Face Image Super-Resolution using Inception Re...
4,ICoICT 2020,Conference proceedings talk,icoict-2020-talk,"Telkom University, Bandung (online)",20-06-24,"Bandung, Indonesia",https://2020.icoict.org/,EDGAN: Disguising Text as Image using Generati...
5,ICoDSA 2020,Conference proceedings talk,icodsa-2020-talk,"Telkom University, Bandung (online)",20-08-05,"Bandung, Indonesia",https://commdis.telkomuniversity.ac.id/icodsa/...,Recognizing Soft Biometric on Pedestrian Using...
6,Asean IVO Forum 2019,Talk,asean-ico-2019-talk,"Crimson Hotel Filinvest City, Manila",19-11-20,"Manila, The Philippines",https://anditya.staff.telkomuniversity.ac.id/a...,Presenting the result of Artificial Intelligen...
7,IAIS Bandung Talk 2020,Talk,iais-bandung-2020-talk,Bandung (online),20-05-15,"Bandung, Indonesia",#,Presenting EDGAN: Disguising Text as Image usi...
8,INACL 2019 Workshop,Talk,inacl-2019-workshop-talk,"Universitas Trisakti, Faculty of Industrial Te...",19-07-18,Jakarta Indonesia,#,Workshop Talk at 5th National Workshop of Indo...
9,DTS/FGA Kominfo August 2019,Tutorial,dts-fga-tutorial-1,"Telkom University, Bandung",19-08-05,"Bandung, Indonesia",https://sites.google.com/telkomuniversity.ac.i...,"Digital Talent Scholarship Tutorial, FGA Machi..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [16]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [17]:
loc_dict = {}

for row, item in talks.iterrows():
    print(row)
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

0
1
2
3
4
5
6
7
8
9
10
11


These files are in the talks directory, one directory below where we're working from.

In [10]:
!ls ../_talks

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
